# Основы программирования на Python. 

## Часть 5: Декораторы

### Задание 1.

Написать функцию, которая на вход будет принимать список объектов, а на выходе возвращать одну строку, где все элементы перечислены через запятую. 

Пример:
<br>&nbsp; ret = merge_list(['Иван', 'Василий', 'Петр'])
<br>&nbsp; print(ret) 
<br>&nbsp; # Результат: 'Иван, Василий, Петр'

In [230]:
# Вариант 1: через цикл while:
def merge_list_1(x):
    ret_str = ''
    i=0
    while i < len(x)-1:
        ret_str += x[i]
        ret_str += ', '
        i += 1
    ret_str += x[len(x)-1]
    return ret_str

In [231]:
ret = merge_list_1(['Иван', 'Василий', 'Петр'])
print(ret)

Иван, Василий, Петр


In [233]:
# Вариант 2: через цикл for с перебором элементов:
def merge_list_2(x):
    ret_str = ''
    for i in x[:-1]:
        ret_str += i
        ret_str += ', '
    ret_str += x[-1]
    return ret_str

In [234]:
ret = merge_list_2(['Иван', 'Василий', 'Петр'])
print(ret)

Иван, Василий, Петр


In [3]:
# Вариант 3: если нужен только вывод результата:
def merge_list_3(x):
    for i in x:
        print(i, end=', ')
    print('\b\b')

In [6]:
merge_list_3(['Иван', 'Василий', 'Петр'])

Иван, Василий, Петр, 


В последнем варианте мы использовали один из необязательных параметров функции print - end. Этот параметр отвечает за последний символ выводимого сообщения. По умолчанию он равен '\n', но можно переопределить его одни из строковых литералов ('\t', '\r', '\b',...) или любой другой строковой переменной.

---

Могут быть ещё варианты с тем, чтобы сразу в цикле собирать всю строку, а затем отрезать лишние символы или некая комбинация этих двух подходов, но это не так принципиально. Вопрос сейчас: какой вариант лучше?

Правильный ответ - ни один из перечисленных.

Почему? Причин 2: первая - при работе с большими данными крайне рекомендуется максимально избегать циклов, так как с точки зрения исполнения кода - они очень затратны. На игрушечных примерах это не так заметно, но сравним эти циклы на более существенном объеме:

In [235]:
%%timeit
ret = merge_list_1(['Иван', 'Василий', 'Петр']*10000)

34.5 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [236]:
%%timeit
ret = merge_list_2(['Иван', 'Василий', 'Петр']*10000)

23.3 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Из полученных результатов мы видим, что в второй вариант реализации работает в среднем быстрее и с меньшим разбросом. Однако, это не предел. Что приводит нас ко второй причине (более значимой) - в python уже реализован инструмент для подобных действий. Это метод join, который формально относится к строковым переменным, но используется в связке со списками. Этот метод позволяет собрать строку из элементов списка, соединив их между собой с помощью заданного значения:

In [237]:
print(', '.join(['Иван', 'Василий', 'Петр']))

Иван, Василий, Петр


Этот метод показывает, насколько удобнее и правильнее использовать уже существующие инструменты, чем создавать свой "велосипед":

In [240]:
%%timeit
', '.join(['Иван', 'Василий', 'Петр']*10000)

1.56 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


---

### Декораторы

Пример выше также демонстрирует нам ещё один удобный, хотя и не столько часто применяющийся механизм - декорирование. Строка %%timeit - представляет из себя ни что иное как декоратор. Суть декораторов, как следует из названия, предоставлять обертку над какой-то функцией. В данном примере эта обертка позволяет запускать нашу функцию несколько раз и считать показатели по времени исполнения. Если совсем коротко, то декоратор - это функция, которая запускается внутри себя другую функцию:

Допустим, у нас есть некая очень полезная функция:

In [154]:
def new_sum(a, b):
    ret = a + b
    return ret

In [155]:
new_sum(3, 5)

8

И мы хотим видеть моменты, когда она запускается. Мы можем прописать это непосредственно в теле функции:

In [156]:
def new_sum(a,b):
    print('Начало выполнения функции')
    ret = a + b
    print('Завершение выполнения функции')
    return ret

In [157]:
new_sum(3, 5)

Начало выполнения функции
Завершение выполнения функции


8

Теперь у нас появилась ещё одна полезная функция, в которой мы тоже хотим видеть аналогичную информацию. Для этого нам придется дублировать куски кода:

In [159]:
def new_mult(a,b):
    print('Начало выполнения функции')
    ret = a * b
    print('Завершение выполнения функции')
    return ret

In [160]:
new_mult(3, 5)

Начало выполнения функции
Завершение выполнения функции


15

Таких функций может становиться больше, а сам лог - сложнее (например, добавится время запуска и окончания). Поддерживать множественный код - не очень удобно. Для этого можно написать над-функцию, которая будет принимать на вход другую функцию (ведь никто не запрещает нам на использовать функцию как аргумент) и оборачивать её в нужные нам логи:

In [161]:
def simple_wrapper(func, *args, **kwargs):
    print('Начало выполнения функции')
    ret=func(*args, **kwargs)
    print('Завершение выполнения функции')
    return ret

Теперь мы можем убрать лишнее из наших функций и использовать функцию-обертку:

In [162]:
def new_sum(a,b):
    ret = a + b
    return ret

def new_mult(a,b):
    ret = a * b
    return ret

In [164]:
new_sum(3, 5), new_mult(3, 5)

(8, 15)

In [165]:
simple_wrapper(new_sum, 3, 5)

Начало выполнения функции
Завершение выполнения функции


8

In [166]:
simple_wrapper(new_mult, 3, 5)

Начало выполнения функции
Завершение выполнения функции


15

Мы почти достигли желаемого результата: каждая функция выполняется с логированием и не нужно прописывать один и тот же функционал много раз. Но у нас появилась новая проблема - читаемость кода ухудшилась: сначала глаз обращает внимание на функцию *simple_wrapper*, а лишь затем на саму полезную нагрузку.

Сделаем ещё один маленький шажок: воспользуемся тем фактом, что функцию (как объект) можно не только принимать на вход, но и возвращать.
Напишем функцию (wrapper), в которой напишем ещё одну функцию (wrapping_func) и будем возвращать эту функцию (wrapping_func) как результат выполнения основной функции (wrapper). А параметром для этой функции (wrapper) станет ещё одна функция, которая будет выполняться внутри функции (wrapping_func). 

Поясним на примере:

In [167]:
def wrapper(func):
    def wrapping_func(*args, **kwargs):
        print('Начало выполнения функции')
        ret = func(*args, **kwargs)
        print('Завершение выполнения функции')
        return ret
    return wrapping_func

Теперь можно сделать следующее:

In [168]:
new_mult = wrapper(new_mult)

In [169]:
new_mult(3, 5)

Начало выполнения функции
Завершение выполнения функции


15

Теперь мы достигли ровно того, чего хотели: мы вызываем функцию по её собственному имени, она возвращает правильный результат, да к тому же ещё и обернута в наш функционал по логированию. 

Как же это произошло? Если посмотреть внимательно, то можно заметить, что мы только что сделали финт ушами - подменили одну функцию другой:

In [170]:
# так было сначала
def new_mult(a,b):
    ret = a * b
    return ret

print(new_mult(3, 5))
print(f'Имя функции - {new_mult.__name__}')

15
Имя функции - new_mult


In [171]:
# Так стало после декоратора:
new_mult = wrapper(new_mult)
print(new_mult(3, 5))
print(f'Имя функции - {new_mult.__name__}')

Начало выполнения функции
Завершение выполнения функции
15
Имя функции - wrapping_func


Главное не увлечься и не завернуть функцию ещё раз (ещё много-много раз) в декоратор:

In [172]:
# Так делать не стоит. Одного раза вполне достаточно:
new_mult = wrapper(new_mult)
print(new_mult(3, 5))
print(f'Имя функции - {new_mult.__name__}')

Начало выполнения функции
Начало выполнения функции
Завершение выполнения функции
Завершение выполнения функции
15
Имя функции - wrapping_func


По сути, это и есть декоратор. Можно пользоваться конструкцией такого вида, но в Python есть альтернативная форма записи, специально для декораторов. Она позволит избежать неприятных коллизий со множественным декорированием:

In [173]:
new_sum(3, 5)

8

In [174]:
@wrapper
def new_sum(a,b):
    ret = a + b
    return ret

In [175]:
new_sum(3, 5)

Начало выполнения функции
Завершение выполнения функции


8

Как было показано ранее, декораторы не обязательно должны быть оформлены именно в таком виде, но подобная запись облегчает понимание кода. Такие подходы в программировании называются "синтаксическим сахаром" - они не прибавляют никакой функциональности, но делают код более доступным к пониманию.

Декораторы - мощный инструмент, который обычно используется для отладки, обработки ошибок или, как в данном случае, для сбора статистики. Однако, это отдельная тема и в данном случае затронута больше для общего понимания.

---

### Задание 2.

Написать декоратор для функций, который будет сохранять все параметры, с которыми запускаются функции. 

Например: "Запущена функция new_sum c параметрами: a = 3, b = 4. Получен результат 17"

In [82]:
def wrapper(func):
    def wrapping_func(*args, **kwargs):
        ret = f'Запущена функция {func.__name__}'
        param = []
        if args:
            param.append(f'{[i for i in args]}')
        if kwargs:
            param.extend([f'{i} = {j}' for i, j in kwargs.items()])
        if len(param)>0:
            ret += ' c параметрами: ' + ', '.join(param)
        ret += '.'
        r = func(*args, **kwargs)
        ret += f' Получен результат {r}'
        print(ret)
        return r
    return wrapping_func

In [83]:
@wrapper
def new_sum(a, b, *args, c=10):
    return a+b+sum([i for i in args]) + c

@wrapper
def new_mult(a, b):
    return a*b

In [84]:
new_sum(a=3, b=4), new_mult(1,11)

Запущена функция new_sum c параметрами: a = 3, b = 4. Получен результат 17
Запущена функция new_mult c параметрами: [1, 11]. Получен результат 11


(17, 11)